In [125]:
# 필요한 라이브러리 설치 및 임포트

import pandas as pd
pd.options.display.max_columns = None

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import recall_score as recall
import joblib

# [Step 3] 머신러닝 모델링1

#### **<span style="color:blue">[3-1] 학습 및 테스트 데이터 로딩</span>**

In [137]:
# train_x.csv / train_y.csv / test_x.csv / test_y.csv 4개의 데이터 로딩
x_train = pd.read_csv("train_x.csv", sep=',', encoding = 'utf-8' )
y_train = pd.read_csv("train_y.csv", sep=',', encoding = 'utf-8' )
x_test = pd.read_csv("test_x.csv", sep = ',', encoding = 'utf-8')
y_test = pd.read_csv("test_y.csv", sep = ',', encoding = 'utf-8')


In [138]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759 entries, 0 to 8758
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   측정소코드      8759 non-null   int64  
 1   측정일시       8759 non-null   int64  
 2   SO2        8759 non-null   float64
 3   CO         8759 non-null   float64
 4   O3         8759 non-null   float64
 5   NO2        8759 non-null   float64
 6   PM10       8759 non-null   float64
 7   PM25       8759 non-null   float64
 8   지점         8759 non-null   int64  
 9   기온(°C)     8759 non-null   float64
 10  풍속(m/s)    8759 non-null   float64
 11  풍향(16방위)   8759 non-null   float64
 12  습도(%)      8759 non-null   int64  
 13  month      8759 non-null   int64  
 14  day        8759 non-null   int64  
 15  hour       8759 non-null   int64  
 16  PM10_lag1  8759 non-null   float64
dtypes: float64(10), int64(7)
memory usage: 1.1 MB


In [139]:
x_train.head()

,측정소코드,측정일시,SO2,CO,O3,NO2,PM10,PM25,지점,기온(°C),풍속(m/s),풍향(16방위),습도(%),month,day,hour,PM10_lag1
0,111123,2021010102,0.002,0.6,0.018,0.020,25.0,14.0,108,-8.7,2.4,270.0,68,1,1,1,38.786049
1,111123,2021010103,0.002,0.6,0.013,0.025,27.0,16.0,108,-9.1,1.6,270.0,69,1,1,2,38.786049
2,111123,2021010104,0.003,0.6,0.011,0.027,23.0,13.0,108,-9.3,1.1,250.0,70,1,1,3,38.786049
3,111123,2021010105,0.003,0.6,0.008,0.032,24.0,14.0,108,-9.3,0.3,0.0,71,1,1,4,38.786049
4,111123,2021010106,0.002,0.7,0.003,0.037,26.0,16.0,108,-9.7,1.9,20.0,72,1,1,5,38.786049


---

#### **<span style="color:blue">[3-2] 모델링: LinearRegression</span>**

* Train과 Test로 나눈 데이터를 기준으로 LinearRegression 모델링을 진행하고 평가를 해주세요.
* 그리고 모델 파일을 pkl로 저장해주세요.
* 성능지표 : MSE, R-squared Score(소수점 5째자리까지)

In [144]:
# LinearRegression 코드를 작성하세요.
model1 = LinearRegression()
model1.fit(x_train,y_train)



LinearRegression()

In [182]:
cv_score = cross_val_score(model1, x_train, y_train, cv=10)
# 확인
print(cv_score)
print('평균:', cv_score.mean())
print('표준편차:', cv_score.std())

[0.91400637 0.95732779 0.91834094 0.97670724 0.90088679 0.82221894
 0.91735874 0.91664425 0.96192862 0.94703641]
평균: 0.9232456104876524
표준편차: 0.04120203222742946


In [145]:
# predict로 예측하여 y_pred_LR에 저장해보세요.
y_pred_LR=model1.predict(x_test)


In [146]:
# test_y 데이터와 y_pred_LR 데이터로 성능을 평가하여 출력해보세요.
# 성능지표는 mse와 r2를 이용하세요.
print('MSE:',mse(y_test,y_pred_LR).round(5))
print('MAE:',mae(y_test,y_pred_LR).round(5))
print('R2:',r2_score(y_test,y_pred_LR).round(5))


MSE: 60.06407
MAE: 5.93447
R2: 0.89117


In [147]:
# 학습한 모델을 pkl 파일로 저장해보세요.
joblib.dump(model1, './LinearRegression_model.pkl')



['./LinearRegression_model.pkl']

# [Step 4] 머신러닝 모델링2

#### **<span style="color:blue">(선택 수행)[4-1] 모델링: 랜덤포레스트</span>**

* 랜덤포레스트: 배깅의 일종으로 의사결정나무(Decision Tree) 여러 개를 모아서 숲을 랜덤으로 구성하고 이를 종합해서 최종 모델을 산출하는 기법
* Train과 Test로 나눈 데이터를 기준으로 렌덤포레스트로 모델을 학습을 진행하고 평가를 해주세요.
* 그리고 모델 파일을 pkl로 저장해주세요.
* 성능지표 : MSE, R-squared Score(소수점 5째자리까지)

In [153]:
import matplotlib.pyplot as plt

In [171]:
# 아래에 필요한 코드를 작성하고 결과를 확인합니다.
model2 = RFR(max_depth=5, random_state=1)
model2.fit(x_train, y_train)

C:\Users\user\AppData\Local\Temp\ipykernel_7936\3467709863.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model2.fit(x_train, y_train)


RandomForestRegressor(max_depth=5, random_state=1)

In [180]:
# Test 데이터로 성능을 평가해보세요.
y_pred_RF=model2.predict(x_test)
print('MSE:',mse(y_test,y_pred_RF).round(5))
print('R2:',r2_score(y_test,y_pred_RF).round(5))



MSE: 41.47105
R2: 0.92486


In [173]:
# 학습한 모델을 파일로 저장해보세요.
joblib.dump(model2, './RandomForestRegressor_model.pkl')


['./RandomForestRegressor_model.pkl']

In [176]:
# (다음 진행은 선택입니다)
# 렌덤포레스트로 학습한 모델의 feature_importances를 구해보세요.
# 확인할 수 있는 내용으로 우리 모델에서의 인사이트를 정리해보세요.
# Feature의 중요도 확인
print()
model2.feature_importances_

['측정소코드', '측정일시', 'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25', '지점', '기온(°C)', '풍속(m/s)', '풍향(16방위)', '습도(%)', 'month', 'day', 'hour', 'PM10_lag1']


array([0.00000000e+00, 1.33512063e-04, 3.50932769e-04, 9.75542889e-04,
       1.93010717e-03, 4.78768900e-04, 9.63574960e-01, 2.17624032e-02,
       0.00000000e+00, 7.74012001e-04, 5.15435075e-04, 5.32622663e-04,
       2.30247087e-03, 6.80815547e-05, 1.27885822e-03, 3.84223818e-03,
       1.48005463e-03])

In [175]:
# (선택) 확인할 수 있는 인사이트
# 1. hour - 습도 -  pm10_lang1 - day
# 2.
# 3.

#### **<span style="color:blue">(선택 수행)[4-2] 모델링: GradientBoosting</span>**

* GradientBoosting: 앞선 모델의 에러를 다음 모델의 예측 값으로 활용하면서 가중치 업데이트 하는데 경사하강법(Gradient Descent)를 활용해서 최적 모델을 만드는 기법
* Train과 Test로 나눈 데이터를 기준으로 그라디언트부스팅으로 모델을 학습을 진행하고 평가를 해주세요.
* 그리고 모델 파일을 pkl로 저장해주세요.
* 성능지표 : RMSE, R-squared Score(소수점 5째자리까지)

In [160]:
# 아래에 필요한 코드를 작성하고 결과를 확인합니다.
# 기본 모델 선언
model_lgb = LGBMRegressor(random_state=1, verbose=-100)

# 파라미터 지정
  # max_depth: range(1, 21)
param = {'max_depth': range(1, 21)}

# 모델 선언
model3 = GridSearchCV(model_lgb,
                     param,
                     cv=5,
                     scoring='r2')


In [161]:
model3.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=LGBMRegressor(random_state=1, verbose=-100),
             param_grid={'max_depth': range(1, 21)}, scoring='r2')

In [159]:
# Test 데이터로 성능을 평가해보세요.
print('MSE:',mse(y_test,y_pred_LR).round(5))
print('R2:',r2_score(y_test,y_pred_LR).round(5))



MSE: 60.06407
R2: 0.89117


In [129]:
print(model.best_params_)
print(model.best_score_)

{'max_depth': 4}
0.8193480246537412


In [14]:
# 학습한 모델을 파일로 저장해보세요.

joblib.dump(model, './GradientBoosting_model.pkl')


In [15]:
# (다음 진행은 선택입니다)
# 그라디언트부스팅으로 학습한 모델의 feature_importances를 구해보세요.
# 확인할 수 있는 내용으로 우리 모델에서의 인사이트를 정리해보세요.
# Feature의 중요도 확인




In [16]:
# 확인할 수 있는 인사이트
# 1.
# 2.
# 3.

#### **<span style="color:blue">(선택 수행)[4-3] 모델링: Self Choice Model</span>**

* Self Choice Model: 앞선 교육과정에서 배운 머신러닝 모델 중 어떤 것이든 좋습니다. 원하는 모델을 선택해서 학습을 시켜보세요.
* Train과 Test로 나눈 데이터를 기준으로 Self Choice Model로 모델을 학습을 진행하고 평가를 해주세요.
* 그리고 모델 파일을 pkl로 저장해주세요.
* 성능지표 : RMSE, R-squared Score(소수점 5째자리까지)

In [132]:
# 아래에 필요한 코드를 작성하고 결과를 확인합니다.
model = DTR(max_depth=4,random_state=1)
model.fit(x_train,y_train)


DecisionTreeRegressor(max_depth=4, random_state=1)

In [133]:
# Test 데이터로 성능을 평가해보세요.

y_pred_DT=model.predict(x_test)


In [134]:
# 학습한 모델을 파일로 저장해보세요.

print('MSE:',mse(y_test,y_pred_DT).round(5))
print('R2:',r2_score(y_test,y_pred_DT).round(5))



MSE: 57.67688
R2: 0.8955


In [184]:
joblib.dump(model, './DecisionTreeRegressor_model.pkl')

['./DecisionTreeRegressor_model.pkl']

In [ ]:
model = KNeighborsClassifier(n_neighbors=5)

In [20]:
# (다음 진행은 선택입니다)
# 그라디언트부스팅으로 학습한 모델의 feature_importances를 구해보세요.
# 확인할 수 있는 내용으로 우리 모델에서의 인사이트를 정리해보세요.
# Feature의 중요도 확인



In [21]:
# Feature의 중요도 확인





In [22]:
# 확인할 수 있는 인사이트
# 1.
# 2.
# 3.

#### **<span style="color:blue">(선택 수행)[4-4] 머신러닝 모델에 대해 성능 최적화 진행</span>**

* 위 머신러닝 모델들에 대해 성능 최적화를 진행해보세요.

In [130]:
# 아래에 필요한 코드를 작성하고 결과를 확인합니다.
plt.figure(figsize=(6, 5))
plt.barh(list(x), model.best_estimator_.feature_importances_)
plt.show()


NameError: name 'plt' is not defined